In [1]:
import numpy as np
import pandas as pd 
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import re
import nltk


In [9]:

products = pd.read_csv(r'.\Recommeder\Data\flipkart_com-ecommerce_sample.csv')
print(products.shape ,len(products['product_name'].unique()),len(products['uniq_id'].unique()))

transactions_path = r".\Recommeder\Data\transaction.csv"

transaction_table = pd.read_csv(transactions_path)


(20000, 15) 12676 20000


In [4]:
def getPid(pname):
    return products[products['product_name'] == pname]['uniq_id'].values[0]

# print(getPid('Urthn Alloy Necklace'))



In [6]:

products['description'] = products['description'].fillna('')
corpus = products['description'] + " " + products['product_name']
corpus = corpus.tolist()

corpus = [re.sub(r'\W', ' ', str(x)) for x in corpus]
corpus = [re.sub(r'\s+[a-zA-Z]\s+', ' ', x) for x in corpus]
corpus = [re.sub(r'\^[a-zA-Z]\s+', ' ', x) for x in corpus]
corpus = [re.sub(r'\s+', ' ', x, flags=re.I) for x in corpus]
corpus = [x.lower() for x in corpus]

# print(corpus[0])
corpus = [x.split() for x in corpus]
print(corpus[0])

['key', 'features', 'of', 'alisha', 'solid', 'women', 'cycling', 'shorts', 'cotton', 'lycra', 'navy', 'red', 'navy', 'specifications', 'of', 'alisha', 'solid', 'women', 'cycling', 'shorts', 'shorts', 'details', 'number', 'of', 'contents', 'in', 'sales', 'package', 'pack', 'of', '3', 'fabric', 'cotton', 'lycra', 'type', 'cycling', 'shorts', 'general', 'details', 'pattern', 'solid', 'ideal', 'for', 'women', 'fabric', 'care', 'gentle', 'machine', 'wash', 'in', 'lukewarm', 'water', 'do', 'not', 'bleach', 'additional', 'details', 'style', 'code', 'altht_3p_21', 'in', 'the', 'box', '3', 'shorts', 'alisha', 'solid', 'women', 'cycling', 'shorts']


In [11]:
# generating sequence of products bought by user

new_transaction_table = transaction_table.groupby('UID')['PID'].apply(list).reset_index(name='sequence')

In [43]:
# save new_transaction_table as csv
new_transaction_table.to_csv(r'.\Recommeder\Data\new_transaction_table.csv', index=False)


In [26]:

class Prod2VecRecommender():
    def __init__(self, min_count=2, size=100, window=5, decay_alpha=0.9, workers=4):

        super(Prod2VecRecommender, self).__init__()
        self.min_count = min_count
        self.size = size
        self.window = window
        self.decay_alpha = decay_alpha
        self.workers = workers

    def __str__(self):
        return 'Prod2VecRecommender(min_count={min_count}, ' \
               'size={size}, ' \
               'window={window}, ' \
               'decay_alpha={decay_alpha}, ' \
               'workers={workers})'.format(**self.__dict__)

    def fit(self, train_data):
        sequences = train_data['sequence']
        self.model = Word2Vec(sentences=sequences, vector_size=self.size, window=self.window, min_count=self.min_count, workers=self.workers, sg=0)
        self.model.train(sequences, total_examples=len(sequences), epochs=100)
        return self



In [27]:
prod2vecRecom = Prod2VecRecommender()
prod2vecRecom.fit(new_transaction_table)



In [34]:

pidToProductSeries = pd.Series(products['product_name'].values,index=products['uniq_id']).to_dict()

# pid to  product metadata dictionary 
# metadata = {pid : [product_name, description, product_category_tree ,brand,product_url ,image ]
# pidToProductMetadata = pd.Series(products[['product_name','description','product_category_tree','brand','product_url','image']].values,index=products['uniq_id']).to_dict()
pidToProductMetadata = {}
for i in range(len(products)):
    pidToProductMetadata[products['uniq_id'][i]] = [products['product_name'][i],products['description'][i],products['product_category_tree'][i],products['brand'][i],products['product_url'][i],products['image'][i]]







In [39]:
# print(pidToProductMetadata['bcb291159f10a43cb08908bd2bdd70c5'])

["Tia by Ten on Ten Styla Women's Full Coverage Bra", "Tia by Ten on Ten Styla Women's Full Coverage Bra - Buy Brown Tia by Ten on Ten Styla Women's Full Coverage Bra For Only Rs. 399 Online in India. Shop Online For Apparels. Huge Collection of Branded Clothes Only at Flipkart.com", '["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Bras >> Tia by Ten on Ten Bras"]', 'Brown', 'http://www.flipkart.com/tia-ten-styla-women-s-full-coverage-bra/p/itmebhchaxd8nn2h?pid=BRAEBHCHY6VVRNRQ', '["http://img5a.flixcart.com/image/bra/n/r/q/wt-br-tia-by-ten-on-ten-36a-original-imaebh64zk4gmwmg.jpeg", "http://img6a.flixcart.com/image/bra/n/r/q/wt-br-tia-by-ten-on-ten-36a-original-imaebh64zk4gmwmg.jpeg", "http://img6a.flixcart.com/image/bra/n/r/q/wt-br-tia-by-ten-on-ten-36b-original-imaebh64qrrqdjyr.jpeg", "http://img5a.flixcart.com/image/bra/n/r/q/wt-br-tia-by-ten-on-ten-34b-original-imaebh64qzqaugdv.jpeg", "http://img6a.flixcart.com/image/bra/n/r/q/wt-br-tia-by-ten-on-ten-36a-original-

In [32]:
products.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [29]:
pidToidx = pd.Series(products.index,index=products['uniq_id']).drop_duplicates()

In [45]:

userId = "2e112284-8013-430d-b784-8f1808dd4e76"

def getreccomend(userId):
    reccomdations = []
    # Past Purchases
    pastPurchases = new_transaction_table[new_transaction_table['UID'] == userId]['sequence'].values

    # print("Past purchases: ", len(pastPurchases[0]))
    # Print name of products bought by user
    # pastProductNames = []
    # for i in pastPurchases[0]:
    #     pastProductNames.append(pidToProductSeries[i])

    # print("Products bought by user: ", len(pastProductNames))
    # print(pastProductNames)


    reccomdations = prod2vecRecom.model.wv.most_similar(positive=pastPurchases[0], topn=10)

    # print("\nTop Recommended products are: \n")
    # print("Product Recommended to the user: ", len(reccomdations))
    # print(reccomdations)

    reccomdation = []
    for i in reccomdations:
        # print(pidToProductSeries[i[0]])
        reccomdation.append(pidToProductMetadata[i[0]])
    # print(reccomdation)
    return reccomdation
    # print(reccomdation)
print(getreccomend(userId))

[["Tia by Ten on Ten Styla Women's Full Coverage Bra", "Tia by Ten on Ten Styla Women's Full Coverage Bra - Buy Brown Tia by Ten on Ten Styla Women's Full Coverage Bra For Only Rs. 399 Online in India. Shop Online For Apparels. Huge Collection of Branded Clothes Only at Flipkart.com", '["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Bras >> Tia by Ten on Ten Bras"]', 'Brown', 'http://www.flipkart.com/tia-ten-styla-women-s-full-coverage-bra/p/itmebhchaxd8nn2h?pid=BRAEBHCHY6VVRNRQ', '["http://img5a.flixcart.com/image/bra/n/r/q/wt-br-tia-by-ten-on-ten-36a-original-imaebh64zk4gmwmg.jpeg", "http://img6a.flixcart.com/image/bra/n/r/q/wt-br-tia-by-ten-on-ten-36a-original-imaebh64zk4gmwmg.jpeg", "http://img6a.flixcart.com/image/bra/n/r/q/wt-br-tia-by-ten-on-ten-36b-original-imaebh64qrrqdjyr.jpeg", "http://img5a.flixcart.com/image/bra/n/r/q/wt-br-tia-by-ten-on-ten-34b-original-imaebh64qzqaugdv.jpeg", "http://img6a.flixcart.com/image/bra/n/r/q/wt-br-tia-by-ten-on-ten-36a-original

In [46]:

import pickle
with open('prod2vec.pkl', 'wb') as f:
    pickle.dump(prod2vecRecom, f)
